In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os
import pickle, glob

from cal_methods import HistogramBinning, TemperatureScaling
from betacal import BetaCalibration
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LogisticRegression

from unpickle_probs import unpickle_probs
from evaluation import calibrationError, softmax

%matplotlib inline

In [2]:
def rel_diagram_sub(accs, confs, bins, ax, M = 10, name = "Reliability Diagram", xname = "", yname=""):
    def autolabel(rects):
        for idx, rect in enumerate(rects):
            height = rect.get_height()
            ax.text(rect.get_x() + rect.get_width() / 2., 1.05 * height,
                    bins[idx],
                    ha='center', va='bottom', rotation=0)

    acc_conf = np.column_stack([accs,confs])
    acc_conf.sort(axis=1)
    outputs = acc_conf[:, 0]
    gap = acc_conf[:, 1]

    bin_size = 1/M
    positions = np.arange(0+bin_size/2, 1+bin_size/2, bin_size)

    # Plot gap first, so its below everything
    gap_plt = ax.bar(positions, gap, width = bin_size, edgecolor = "red", color = "red", alpha = 0.3, label="Gap", linewidth=2, zorder=2)

    autolabel(gap_plt)

    # Next add error lines
    #for i in range(M):
        #plt.plot([i/M,1], [0, (M-i)/M], color = "red", alpha=0.5, zorder=1)

    #Bars with outputs
    output_plt = ax.bar(positions, outputs, width = bin_size, edgecolor = "black", color = "blue", label="Outputs", zorder = 3)

    # Line plot with center line.
    ax.set_aspect('equal')
    ax.plot([0,1], [0,1], linestyle = "--")
    ax.legend(handles = [gap_plt, output_plt])
    ax.set_xlim(0,1)
    ax.set_ylim(0,1)
    ax.set_title(name, fontsize=24)
    ax.set_xlabel(xname, fontsize=22, color = "black")
    ax.set_ylabel(yname, fontsize=22, color = "black")


In [3]:
def get_pred_conf(y_probs, normalize=False):
    y_preds = np.argmax(y_probs, axis=1)  # Take maximum confidence as prediction

    if normalize:
        y_confs = np.max(y_probs, axis=1) / np.sum(y_probs, axis=1)
    else:
        y_confs = np.max(y_probs, axis=1)  # Take only maximum confidence

    return y_preds, y_confs

In [10]:
def calibrate_on_samples(method, path, file, M=15, approach="single", m_kwargs={}):
    bin_size = 1 / M

    FILE_PATH = os.path.join(path, file)
    (y_logits_val, y_val), (y_logits_test, y_test) = unpickle_probs(FILE_PATH)

    y_probs_val = softmax(y_logits_val)[:, :10]  # Softmax logits
    y_probs_test = softmax(y_logits_test)[:, :10]
    
    if approach == "single":
        model = dict()
        K = y_probs_test.shape[1]

        # Go through all the classes
        for k in range(K):
            # Prep class labels (1 fixed true class, 0 other classes)
            y_cal = np.array(y_val == k, dtype="int")[:, 0]

            # Train model
            model[k] = method(**m_kwargs)
            model[k].fit(y_probs_val[:, k], y_cal)  # Get only one column with probs for given class "k"

    else:
        model = method(**m_kwargs)
        model.fit(y_logits_val, y_val)

    return model

In [11]:
def cal_res(model, path, file, M=15, approach="single"):
    bin_size = 1 / M

    FILE_PATH = os.path.join(path, file)
    (y_logits_val, y_val), (y_logits_test, y_test) = unpickle_probs(FILE_PATH)

    y_probs_val = softmax(y_logits_val)[:, :10]  # Softmax logits
    y_probs_test = softmax(y_logits_test)[:, :10]

    if approach == "single":

        K = y_probs_test.shape[1]

        # Go through all the classes
        for k in range(K):
            # Prep class labels (1 fixed true class, 0 other classes)
            y_cal = np.array(y_val == k, dtype="int")[:, 0]

            y_probs_val[:, k] = model[k].predict(y_probs_val[:, k])  # Predict new values based on the fittting
            y_probs_test[:, k] = model[k].predict(y_probs_test[:, k])

            # Replace NaN with 0, as it should be close to zero  # TODO is it needed?
            idx_nan = np.where(np.isnan(y_probs_test))
            y_probs_test[idx_nan] = 0

            idx_nan = np.where(np.isnan(y_probs_val))
            y_probs_val[idx_nan] = 0

            y_preds_val, y_confs_val = get_pred_conf(y_probs_val, normalize=True)
            y_preds_test, y_confs_test = get_pred_conf(y_probs_test, normalize=True)

    else:
        y_probs_val = model.predict(y_logits_val)
        y_probs_test = model.predict(y_logits_test)

        y_preds_val, y_confs_val = get_pred_conf(y_probs_val, normalize=False)
        y_preds_test, y_confs_test = get_pred_conf(y_probs_test, normalize=False)
        
    errors_val = calibrationError(y_confs_val, y_preds_val, y_val, bin_size=bin_size)
    _, _, bin_info_val = errors_val.calculate_errors()
    
    errors_test = calibrationError(y_confs_test, y_preds_test, y_test, bin_size=bin_size)
    _, _, bin_info_test = errors_test.calculate_errors()
    
    return (bin_info_test["accuracies"], bin_info_test["confidences"], bin_info_test["bin_lengths"]), (bin_info_val["accuracies"], bin_info_val["confidences"], bin_info_val["bin_lengths"])


In [12]:
def get_uncalibrated_res(path, file, M=15):
    bin_size = 1 / M

    FILE_PATH = os.path.join(path, file)
    (y_logits_val, y_val), (y_logits_test, y_test) = unpickle_probs(FILE_PATH)

    y_probs_test = softmax(y_logits_test)
    y_preds_test, y_confs_test = get_pred_conf(y_probs_test, normalize=False)
    
    errors = calibrationError(y_confs_test, y_preds_test, y_test, bin_size=bin_size)
    _, _, bin_info = errors.calculate_errors()
    
    return bin_info["accuracies"], bin_info["confidences"], bin_info["bin_lengths"]


In [13]:
def gen_plots(files, plot_names=[], M=15, val_set=False):
    pickle_path = os.path.join("..", "pickle_dump")
    out_path = os.path.join("..", "reliability_plots")
    if val_set:  # Plot Reliability diagrams for validation set
        k = 1
    else:
        k = 0
        
    original_file = [s for s in files if "sample" in s][0]
    model_ts = calibrate_on_samples(TemperatureScaling, PATH, original_file, M, "multi")
    model_hb = calibrate_on_samples(HistogramBinning, PATH, original_file, M, "single", {'M': M})
    model_iso = calibrate_on_samples(IsotonicRegression, PATH, original_file, M, "single", {'y_min': 0, 'y_max': 1})
    model_beta = calibrate_on_samples(BetaCalibration, PATH, original_file, M, "single", {'parameters': "abm"})
    
    for i, file in enumerate(files):

        bin_info_uncal = get_uncalibrated_res(PATH, file, M)

        accs_confs = []
        
        accs_confs.append(cal_res(model_ts, PATH, file, M, "multi"))
        accs_confs.append(cal_res(model_hb, PATH, file, M, "single"))
        accs_confs.append(cal_res(model_iso, PATH, file, M, "single"))
        accs_confs.append(cal_res(model_beta, PATH, file, M, "single"))

        with open(os.path.join(pickle_path, plot_names[i] + "_bin_info.p"), "wb") as f:
            pickle.dump(accs_confs, f)

        plt.style.use('ggplot')
        fig, ax = plt.subplots(nrows=1, ncols=5, figsize=(22.5, 4), sharex='col', sharey='row')
        names = [" (Uncal)", " (Temp)", " (Histo)", " (Iso)", " (Beta)"]

        # Uncalibrated information
        rel_diagram_sub(bin_info_uncal[0], bin_info_uncal[1], bin_info_uncal[2], ax[0], M=M,
                        name="\n".join(plot_names[i].split()) + names[0], xname="Confidence")

        for j in range(4):
            rel_diagram_sub(accs_confs[j][k][0], accs_confs[j][k][1], accs_confs[j][k][2], ax[j + 1], M=M,
                            name="\n".join(plot_names[i].split()) + names[j + 1], xname="Confidence")

        ax[0].set_ylabel("Accuracy", color="black")

        for ax_temp in ax:
            plt.setp(ax_temp.get_xticklabels(), rotation='horizontal', fontsize=18)
            plt.setp(ax_temp.get_yticklabels(), fontsize=18)

        plt.savefig(os.path.join(out_path, "_".join(plot_names[i].split()) + ".pdf"), format='pdf', dpi=1000, bbox_inches='tight',
                pad_inches=0.2)
        plt.show()


In [14]:
seeds = [32, 100] #, 242, 376, 498]
filter_list = ["sample", "blur", "detail", "edge_enhance", "sharp", "smooth"]
PATH = os.path.join('..', 'logits_context')
#files_original = ["probs_resnet110_original_" + f + "_32_logits.p" for f in filter_list]

files_original = dict()
files_blurdetail = dict()
files_allfilter = dict()

for seed in seeds:
    #files_original[seed] = ["probs_original_" + f + "_" + str(seed) + "_logits.p" for f in filter_list]
    files_blurdetail[seed] = ["probs_resnet101_blurdetail_context_" + f + "_" + str(seed) + "_logits.p" for f in filter_list]
    #files_allfilter[seed] = ["probs_allfilter_" + f + "_" + str(seed) + "_logits.p" for f in filter_list]
    

In [15]:
for seed in seeds:
        print("Generating for seed: {b}".format(b=seed))
        #plot_names = ["ResNet-110- original " + f + "_" + str(seed) for f in filter_list]
        #gen_plots(files_original[seed], plot_names=plot_names, val_set=False)

        plot_names = ["ResNet-101- blurdetail " + f + "_" + str(seed) for f in filter_list]
        gen_plots(files_blurdetail[seed], plot_names=plot_names, val_set=False)

        #plot_names = ["ResNet-110- all_filter " + f + "_" + str(seed) for f in filter_list]
        #gen_plots(files_allfilter[seed], plot_names=plot_names, val_set=False)

Generating for seed: 32


ValueError: y_true and y_pred contain different number of classes 10, 16. Please provide the true labels explicitly through the labels argument. Classes found in y_true: [0 1 2 3 4 5 6 7 8 9]